# DETR模型结构总结

DETR（Detection Transformer）是基于Transformer的端到端目标检测模型，结构包括四部分：

1. CNN主干网络（Backbone）
   - 功能：提取输入图像的初始特征
   - 操作：
     - 采用ResNet（如ResNet-50/101）提取特征，输出深层特征图（如C5层2048×25×34）
     - 通过1×1卷积将通道数从2048降维至256，得到256×H×W的特征图{insert\_element\_0\_}

2. Transformer编码器（Encoder）
   - 功能：对特征进行全局建模，捕捉目标间长距离依赖
   - 操作：
     - 添加固定位置编码（256×H×W），与特征图相加
     - 将特征图展平为序列（长度H×W，如850×256）
     - 通过6层Transformer编码器（含多头自注意力和前馈网络）输出全局特征{insert\_element\_1\_}

3. Transformer解码器（Decoder）
   - 功能：结合全局特征与可学习查询向量生成预测
   - 关键组件：
     - Object Query：100个可学习向量（100×256），指定输出框数量
     - 注意力机制：先自注意力（去冗余），再交叉注意力（关联图像特征）
     - 通过6层解码器输出100×256特征{insert\_element\_2\_}

4. 预测头（Prediction Heads）
   - 功能：将解码器输出转换为检测结果
   - 操作：
     - 分类分支：FFN输出91维向量（COCO类别），经softmax得类别概率
     - 回归分支：FFN输出4维向量（x, y, w, h）表示边界框{insert\_element\_3\_}

# DETR损失计算总结

DETR通过二分图匹配实现预测框与真实框的一对一匹配，损失包括两部分：

1. 二分图匹配（Hungarian算法）
   - 目的：找到100个预测框与真实框的最优匹配
   - 匹配损失（Cost Matrix）：
     cost(i, j) = -𝟙_{c_j≠∅}·p̂_i(c_j) + 𝟙_{c_j≠∅}·ℒ_box(b_j, b̂_i)
     其中：
     - p̂_i(c_j)为预测框i属于类别c_j的概率
     - ℒ_box为边界框损失（L1+GIoU）{insert\_element\_4\_}

2. 最终损失函数（Hungarian Loss）
   - 总损失：
     ℒ_Hungarian(y, ŷ) = Σ[-log p̂_σ(i)(c_i) + 𝟙_{c_i≠∅}·ℒ_box(b_i, b̂_σ(i))]
     包含：
     1. 分类损失：交叉熵损失，计算匹配框的类别误差
     2. 边界框损失：
        ℒ_reg = λ_L1·L1Loss + λ_GIoU·GIoULoss
        - L1Loss：|b̂ - b|的平均值，关注位置精度
        - GIoULoss：1 - GIoU，优化全局重叠度{insert\_element\_5\_}

3. 辅助损失：在每个解码器层后计算损失并累加，加速收敛{insert\_element\_6\_}